In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
df1 = pd.read_csv('data/space_titanic.csv',skipinitialspace=True)

In [31]:
df1.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [32]:
df1.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

    Quitamos las columnas innecesarias: Name, PassengerId y Transported, siendo esta la variable predictora

In [33]:
df1 = df1.drop(['Name','PassengerId'],axis=1)

In [34]:
df1.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [35]:
df2 = df1.copy()

Inputamos valores NaN, primero con los valores de las amenities de RoomService, FoodCourt, ShoppingMall, Spa y VRDeck

In [36]:
from sklearn.impute import KNNImputer
from xgbimputer import XGBImputer
import miceforest as mf

In [37]:
df2_columns = df2[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]

In [38]:
df2 = df2.drop(['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'],axis=1)

In [39]:
df2_columns.head()

,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.0,0.0,0.0,0.0,0.0
1,109.0,9.0,25.0,549.0,44.0
2,43.0,3576.0,0.0,6715.0,49.0
3,0.0,1283.0,371.0,3329.0,193.0
4,303.0,70.0,151.0,565.0,2.0


In [40]:
df2_columns = df2_columns.replace({pd.NA: np.nan})

In [41]:
imputer = KNNImputer(n_neighbors=4, weights="uniform")
result = imputer.fit_transform(df2_columns)

In [42]:
knn_imputed = pd.DataFrame(result,columns=df2_columns.columns)

In [43]:
knn_imputed.isna().sum()

RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64

In [44]:
knn_imputed.shape

(8693, 5)

In [45]:
df3 = df2.join(knn_imputed)

In [46]:
df3.isna().sum()

HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
Transported       0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
dtype: int64

In [47]:
df3.dtypes

HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
Transported        bool
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
dtype: object

In [48]:
df3.replace({'VIP' : {False : 0, True : 1}},inplace=True)
df3.replace({'CryoSleep' : {False : 0, True : 1}},inplace=True)

In [49]:
df3.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,Transported,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,0.0,B/0/P,TRAPPIST-1e,39.0,0.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,0.0,F/0/S,TRAPPIST-1e,24.0,0.0,True,109.0,9.0,25.0,549.0,44.0
2,Europa,0.0,A/0/S,TRAPPIST-1e,58.0,1.0,False,43.0,3576.0,0.0,6715.0,49.0
3,Europa,0.0,A/0/S,TRAPPIST-1e,33.0,0.0,False,0.0,1283.0,371.0,3329.0,193.0
4,Earth,0.0,F/1/S,TRAPPIST-1e,16.0,0.0,True,303.0,70.0,151.0,565.0,2.0


In [50]:
df3['VIP'] = df3['VIP'].astype(bool)
df3['CryoSleep'] = df3['CryoSleep'].astype(bool)

In [51]:
df3.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,Transported,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,True,109.0,9.0,25.0,549.0,44.0
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,False,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,False,0.0,1283.0,371.0,3329.0,193.0
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,True,303.0,70.0,151.0,565.0,2.0


In [52]:
from sklearn.impute import SimpleImputer

In [53]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean') #en base al EDA

In [54]:
df3['Age'] = imp_mean.fit_transform(df3['Age'].values.reshape(-1,1))

In [55]:
df3.isna().sum()

HomePlanet      201
CryoSleep         0
Cabin           199
Destination     182
Age               0
VIP               0
Transported       0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
dtype: int64

In [56]:
imp_mstfq = SimpleImputer(missing_values=np.nan, strategy='most_frequent') #en base al EDA

In [57]:
df3['HomePlanet'] = imp_mstfq.fit_transform(df3['HomePlanet'].values.reshape(-1,1))
df3['Destination'] = imp_mstfq.fit_transform(df3['Destination'].values.reshape(-1,1))

In [58]:
df3 = df3.dropna(subset = ['Cabin'],inplace=False)

In [59]:
df3.isna().sum()

HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
Transported     0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64

In [60]:
df3.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,Transported,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,True,109.0,9.0,25.0,549.0,44.0
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,False,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,False,0.0,1283.0,371.0,3329.0,193.0
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,True,303.0,70.0,151.0,565.0,2.0


In [61]:
df3[['Deck', 'Num', 'Side']] = df3['Cabin'].str.split('/', expand=True)

In [62]:
df3.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,Transported,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,False,0.0,0.0,0.0,0.0,0.0,B,0,P
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,True,109.0,9.0,25.0,549.0,44.0,F,0,S
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,False,43.0,3576.0,0.0,6715.0,49.0,A,0,S
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,False,0.0,1283.0,371.0,3329.0,193.0,A,0,S
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,True,303.0,70.0,151.0,565.0,2.0,F,1,S


In [63]:
df3.isna().sum()

HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
Transported     0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Deck            0
Num             0
Side            0
dtype: int64

In [64]:
df3 = pd.get_dummies(df3, columns=['HomePlanet','Destination'], drop_first=False)

In [65]:
df3 = pd.get_dummies(df3, columns=['Deck','Side'], drop_first=False)

In [66]:
df3.head()

,CryoSleep,Cabin,Age,VIP,Transported,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S
0,False,B/0/P,39.0,False,False,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,1,0
1,False,F/0/S,24.0,False,True,109.0,9.0,25.0,549.0,44.0,...,0,0,0,0,0,1,0,0,0,1
2,False,A/0/S,58.0,True,False,43.0,3576.0,0.0,6715.0,49.0,...,1,0,0,0,0,0,0,0,0,1
3,False,A/0/S,33.0,False,False,0.0,1283.0,371.0,3329.0,193.0,...,1,0,0,0,0,0,0,0,0,1
4,False,F/1/S,16.0,False,True,303.0,70.0,151.0,565.0,2.0,...,0,0,0,0,0,1,0,0,0,1


In [67]:
df3.to_csv(r'data/space_etl2.csv', index=False)

Otro posible dataset (eliminando NaN, f < 0.03)

In [71]:
df4 = df1.copy()

In [72]:
df4.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [73]:
df4 = df4.dropna(subset = ['HomePlanet','Cabin','Destination','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','CryoSleep','VIP','Age'],inplace=False)

In [13]:
df4.isna().sum()

HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
dtype: int64

In [74]:
df4.shape #vemos que el numero de muestras se ha reducido drasticamente ~2000

(6764, 12)

In [75]:
df4 = pd.get_dummies(df4, columns=['HomePlanet','Destination'], drop_first=False)

In [76]:
df4.replace({'VIP' : {False : 0, True : 1}},inplace=True)
df4.replace({'CryoSleep' : {False : 0, True : 1}},inplace=True)

In [77]:
df4.head()

,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,B/0/P,39.0,0,0.0,0.0,0.0,0.0,0.0,False,0,1,0,0,0,1
1,0,F/0/S,24.0,0,109.0,9.0,25.0,549.0,44.0,True,1,0,0,0,0,1
2,0,A/0/S,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,0,1,0,0,0,1
3,0,A/0/S,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,0,1,0,0,0,1
4,0,F/1/S,16.0,0,303.0,70.0,151.0,565.0,2.0,True,1,0,0,0,0,1


In [78]:
df4['VIP'] = df4['VIP'].astype(bool)
df4['CryoSleep'] = df4['CryoSleep'].astype(bool)

In [79]:
df4[['Deck', 'Num', 'Side']] = df4['Cabin'].str.split('/', expand=True)

In [80]:
df4 = pd.get_dummies(df4, columns=['Deck','Side'], drop_first=False)

In [81]:
df4.head()

,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,...,Deck_A,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_P,Side_S
0,False,B/0/P,39.0,False,0.0,0.0,0.0,0.0,0.0,False,...,0,1,0,0,0,0,0,0,1,0
1,False,F/0/S,24.0,False,109.0,9.0,25.0,549.0,44.0,True,...,0,0,0,0,0,1,0,0,0,1
2,False,A/0/S,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,...,1,0,0,0,0,0,0,0,0,1
3,False,A/0/S,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,...,1,0,0,0,0,0,0,0,0,1
4,False,F/1/S,16.0,False,303.0,70.0,151.0,565.0,2.0,True,...,0,0,0,0,0,1,0,0,0,1


In [82]:
df4.to_csv(r'data/space_etl3.csv', index=False)

Otro posible dataset: imputación simple y eliminación de algunos NaN

In [83]:
df5 = df1.copy()

In [84]:
df5.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [85]:
imp_median = SimpleImputer(missing_values=np.nan, strategy='median') #en base al EDA

In [86]:
df5['RoomService'] = imp_median.fit_transform(df5['RoomService'].values.reshape(-1,1))
df5['FoodCourt'] = imp_median.fit_transform(df5['FoodCourt'].values.reshape(-1,1))
df5['ShoppingMall'] = imp_median.fit_transform(df5['ShoppingMall'].values.reshape(-1,1))
df5['Spa'] = imp_median.fit_transform(df5['Spa'].values.reshape(-1,1))
df5['VRDeck'] = imp_median.fit_transform(df5['VRDeck'].values.reshape(-1,1))

In [87]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean') #en base al EDA

In [88]:
df5['Age'] = imp_mean.fit_transform(df5['Age'].values.reshape(-1,1))

In [89]:
df5.isna().sum()

HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age               0
VIP             203
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [90]:
imp_mostfq = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [91]:
df5['CryoSleep'] = imp_mostfq.fit_transform(df5['CryoSleep'].values.reshape(-1,1))
df5['VIP'] = imp_mostfq.fit_transform(df5['VIP'].values.reshape(-1,1))

In [92]:
df5.replace({'VIP' : {False : 0, True : 1}},inplace=True)
df5.replace({'CryoSleep' : {False : 0, True : 1}},inplace=True)
df5['VIP'] = df5['VIP'].astype(bool)
df5['CryoSleep'] = df5['CryoSleep'].astype(bool)

In [93]:
df5.isna().sum()

HomePlanet      201
CryoSleep         0
Cabin           199
Destination     182
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [95]:
df5 = df5.dropna(subset = ['Cabin'],inplace=False)

In [96]:
df5['HomePlanet'] = imp_mostfq.fit_transform(df5['HomePlanet'].values.reshape(-1,1))
df5['Destination'] = imp_mostfq.fit_transform(df5['Destination'].values.reshape(-1,1))

In [97]:
df5.isna().sum()

HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64

In [98]:
df5.shape

(8494, 12)

In [99]:
df5[['Deck', 'Num', 'Side']] = df5['Cabin'].str.split('/', expand=True)

In [100]:
df5.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,0,P
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,0,S
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,0,S
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,0,S
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,1,S


In [101]:
df5 = pd.get_dummies(df5, columns=['Deck','Side','Destination','HomePlanet'], drop_first=False)

In [102]:
df5.head()

,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,...,Deck_G,Deck_T,Side_P,Side_S,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars
0,False,B/0/P,39.0,False,0.0,0.0,0.0,0.0,0.0,False,...,0,0,1,0,0,0,1,0,1,0
1,False,F/0/S,24.0,False,109.0,9.0,25.0,549.0,44.0,True,...,0,0,0,1,0,0,1,1,0,0
2,False,A/0/S,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,...,0,0,0,1,0,0,1,0,1,0
3,False,A/0/S,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,...,0,0,0,1,0,0,1,0,1,0
4,False,F/1/S,16.0,False,303.0,70.0,151.0,565.0,2.0,True,...,0,0,0,1,0,0,1,1,0,0


In [103]:
df5.to_csv(r'data/space_etl4.csv', index=False)